In [6]:
import requests 
import pandas as pd    
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os


In [7]:
load_dotenv(dotenv_path='../erdos_finance_llm\erdos.env')

def get_news(symbols, start_date, end_date,api_key):
    if api_key is None:
        api_key = os.getenv('ALPHA_VANTAGE_API_KEY')
        if api_key is None:
            raise ValueError("API key not found in environment variables")
    if start_date is None:
        start_date = (datetime.now() - timedelta(days=7)).strftime('%Y%m%dT0000')
    else:
        start_date = datetime.strptime(start_date, '%Y-%m-%d').strftime('%Y%m%dT0000')
    if end_date is None:
        end_date = datetime.now().strftime('%Y%m%dT0000')
    else:
        end_date = datetime.strptime(end_date, '%Y-%m-%d').strftime('%Y%m%dT0000')

    news_data = []
    try:
        for symbol in symbols:
            url = f"https://www.alphavantage.co/query"
            params ={
                'function': 'NEWS_SENTIMENT',
                'tickers': symbol,
                'apikey': api_key,
                'time_from': start_date,
                'time_to': end_date,
                'sort': 'LATEST',
                
            }
            response = requests.get(url, params=params)
            data = response.json()
            if 'feed' not in data:
                print(f"No news found for {symbol} in the given date range")
                continue
            for item in data['feed']:
               news = {
                   'symbol': symbol,
                   'title': item['title'],
                   'source': item['source'],
                   'date': datetime.strptime(item['time_published'], '%Y%m%dT%H%M%S'),
                   'summary': item['summary'],
                   'url': item['url'],
               }
               
               if 'ticker_sentiment' in item:
                   for ticker_data in item['ticker_sentiment']:
                       if ticker_data['ticker'] == symbol:
                           news['sentiment'] = ticker_data['ticker_sentiment_label']
                           news['sentiment_score'] = ticker_data['ticker_sentiment_score']
                           break

               news_data.append(news)
                
    except Exception as e:
        print(f"Error fetching news for {symbol}: {e}")
        return pd.DataFrame()
    
    if news_data:
        news_df = pd.DataFrame(news_data)
        news_df = news_df.sort_values(by='date', ascending=False)
        return news_df
    else:
        return pd.DataFrame()

    

                        


<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
C:\Users\zxmat\AppData\Local\Temp\ipykernel_1160\1104685585.py:1: SyntaxWarning: invalid escape sequence '\e'
  load_dotenv(dotenv_path='../erdos_finance_llm\erdos.env')


In [8]:
test_news = get_news(
    symbols=["TSLA", "AAPL"],
    start_date="2025-03-01",
    end_date="2025-03-11",
    api_key=os.getenv('ALPHA_VANTAGE_API_KEY')
)

In [9]:
test_news

,symbol,title,source,date,summary,url,sentiment,sentiment_score
0,TSLA,"Tesla Out, Ford In: City Of Baltimore Goes 'In...",Benzinga,2025-03-10 23:07:12,A large U.S. city pauses a deal with Tesla for...,https://www.benzinga.com/trading-ideas/movers/...,Neutral,0.13002
1,TSLA,"""R-Word"" Fears Send Stock Market Much Lower",Zacks Commentary,2025-03-10 22:13:00,While the tech-heavy Nasdaq and small-cap Russ...,https://www.zacks.com/stock/news/2427989/r-wor...,Neutral,-0.100183
50,AAPL,"""R-Word"" Fears Send Stock Market Much Lower",Zacks Commentary,2025-03-10 22:13:00,While the tech-heavy Nasdaq and small-cap Russ...,https://www.zacks.com/stock/news/2427989/r-wor...,Neutral,-0.100183
2,TSLA,DOGE chief Musk says he's running his business...,CNBC,2025-03-10 21:22:39,Musk said he expected to realize $1 trillion i...,https://www.cnbc.com/2025/03/10/musk-tesla-dog...,Neutral,-0.047195
3,TSLA,Tesla Bloodbath To Continue? Hedge Funder Says...,Benzinga,2025-03-10 20:44:34,Tesla stock fell significantly on Monday with ...,https://www.benzinga.com/trading-ideas/movers/...,Neutral,-0.143642
...,...,...,...,...,...,...,...,...
95,AAPL,Is Franklin U.S. Equity Index ETF ( USPX ) a...,Zacks Commentary,2025-03-05 11:20:08,Smart Beta ETF report for ...,https://www.zacks.com/stock/news/2425682/is-fr...,Neutral,0.060868
96,AAPL,Should SPDR S&P 500 ETF ( SPY ) Be on Your I...,Zacks Commentary,2025-03-05 11:20:07,Style Box ETF report for SPY ...,https://www.zacks.com/stock/news/2425684/shoul...,Neutral,0.071604
97,AAPL,Should iShares Core S&P 500 ETF ( IVV ) Be o...,Zacks Commentary,2025-03-05 11:20:06,Style Box ETF report for ...,https://www.zacks.com/stock/news/2425690/shoul...,Neutral,0.071708
98,AAPL,Should Fidelity Quality Factor ETF ( FQAL ) ...,Zacks Commentary,2025-03-05 11:20:05,Style Box ETF report for ...,https://www.zacks.com/stock/news/2425693/shoul...,Neutral,0.071604


In [28]:
NEWS_ANALYSIS_PROMPT = """
    You are an expert financial analyst.
    I will provide you with a news article related to a stock.
    1. Determine sentiment: Positive, Negative, Neutral.
    2. Summary key points relevant to the stock.
    3. Provide an investment recommendation based on this specific article.
    
    Format your output as :
    {
        "sentiment": "Positive/Negative/Neutral",
        "key_points": [
            "...",
            "..."
        ],
        # "summary": "...",
        "investment_recommendation": "..."
    }
    """

In [21]:
from groq import Groq
import json 
import re

In [30]:
def analyze_news(news_df, api_key=None, model='llama3-8b-8192'):
    if api_key is None:
        print('Api_key None')
    client = Groq(api_key=api_key)
    
    results = []
    
    for _, row in news_df.iterrows():
        article_text = f'Title: {row['title']}\nContent: {row['summary']}'
        
        completion = client.chat.completions.create(
            messages=[
                {"role": "system", "content": NEWS_ANALYSIS_PROMPT},
                {"role": "user", "content": f"Stock: {row['symbol']}\nNews Article:\n{article_text}"}
            ],
            model = model
        )
        analysis = completion.choices[0].message.content
        
        try:
            result = json.loads(analysis)
        except:
            json_match = re.search(r'({.*})', analysis.replace('\n', ''), re.DOTALL)
            if json_match:
                try:
                    result = json.loads(json_match.group(1))
                except:
                    result = {"sentiment": "error", "summary": analysis[:100]}
            else:
                result = {"sentiment": "error", "summary": analysis[:100]}
                
        results.append({
            **row.to_dict(),
            'ai_sentiment': result.get('sentiment', ''),
            
            'ai_recommendation': result.get('investment_recommendation', '')
        })
    
    return pd.DataFrame(results)
                
    

In [31]:
analyze_news(test_news.loc[0:10], api_key=os.getenv('GROQ_API_KEY'), model='llama3-8b-8192')

,symbol,title,source,date,summary,url,sentiment,sentiment_score,ai_sentiment,ai_recommendation
0,TSLA,"Tesla Out, Ford In: City Of Baltimore Goes 'In...",Benzinga,2025-03-10 23:07:12,A large U.S. city pauses a deal with Tesla for...,https://www.benzinga.com/trading-ideas/movers/...,Neutral,0.13002,Negative,Slightly Sell TSLA. This news may further nega...
1,TSLA,"""R-Word"" Fears Send Stock Market Much Lower",Zacks Commentary,2025-03-10 22:13:00,While the tech-heavy Nasdaq and small-cap Russ...,https://www.zacks.com/stock/news/2427989/r-wor...,Neutral,-0.100183,error,
2,AAPL,"""R-Word"" Fears Send Stock Market Much Lower",Zacks Commentary,2025-03-10 22:13:00,While the tech-heavy Nasdaq and small-cap Russ...,https://www.zacks.com/stock/news/2427989/r-wor...,Neutral,-0.100183,error,
3,TSLA,DOGE chief Musk says he's running his business...,CNBC,2025-03-10 21:22:39,Musk said he expected to realize $1 trillion i...,https://www.cnbc.com/2025/03/10/musk-tesla-dog...,Neutral,-0.047195,error,
4,TSLA,Tesla Bloodbath To Continue? Hedge Funder Says...,Benzinga,2025-03-10 20:44:34,Tesla stock fell significantly on Monday with ...,https://www.benzinga.com/trading-ideas/movers/...,Neutral,-0.143642,Negative,SELL/THEM - The negative sentiment and bearish...
5,TSLA,RC BREAKING NEWS: A Securities Fraud Class Act...,Benzinga,2025-03-10 20:34:10,"NEW YORK, March 10, 2025 ( GLOBE NEWSWIRE ) --...",https://www.benzinga.com/pressreleases/25/03/g...,Neutral,0.141373,Neutral,"Based on this news article, we do not recommen..."
6,TSLA,Tesla Stock Suffers Worst Day In 5 Years After...,Benzinga,2025-03-10 20:06:17,Tesla's sales in Europe fell by about 45% in J...,https://www.benzinga.com/trading-ideas/movers/...,Neutral,-0.138408,Negative,Given the recent decline in sales in Europe an...
7,AAPL,The Reality of Trump's Tariffs,Motley Fool,2025-03-10 19:46:00,"In this podcast, Motley Fool analyst Jason Mos...",https://www.fool.com/investing/2025/03/10/the-...,Neutral,0.064205,Neutral,"Hold or neutral on AAPL stock, as the article ..."
8,TSLA,What's Going On With Tesla Shares Monday? - Te...,Benzinga,2025-03-10 19:07:35,Tesla's sales in Europe fell by about 45% in J...,https://www.benzinga.com/25/03/44232237/whats-...,Somewhat-Bearish,-0.230503,Negative,Caution is advised. Tesla's sales decline in E...
9,TSLA,5 Top-Ranked S&P 500 Stocks to Buy at a Bargai...,Zacks Commentary,2025-03-10 19:00:00,The S&P 500 logs the worst week since Septembe...,https://www.zacks.com/stock/news/2427947/5-top...,Neutral,0.019829,error,
